# Statewide Data with Crash Severity as Target

## Imports and Setup

In [31]:
# Initial imports
import pandas as pd
import numpy as np
import psycopg2 
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [32]:
# Load in password
from secret import secret, database, username, host

# Establish connection to database
conn = psycopg2.connect(
    host = host,
    database = database,
    user =username,
    password = secret
)

In [33]:
# Load the data
query = 'SELECT * FROM clark_co_traffic'
traffic_df = pd.read_sql_query(query, conn)

/var/folders/f3/5zcd9_1j7d3546ws7g9t5fw00000gp/T/ipykernel_11950/3027837082.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  traffic_df = pd.read_sql_query(query, conn)


## Preprocess Data

In [34]:
# Drop unnecessary columns
traffic_df.drop(columns = ['objectid','injury_type','accident_rec_num','primary_street', 'secondary_street', 'fatalities', 'injured','property_damage_only' ], inplace = True)

In [35]:
traffic_df.columns

Index(['x', 'y', 'crash_severity', 'county', 'crash_date', 'crash_year',
       'crash_time', 'weather', 'crash_type', 'total_vehicles', 'v1_type',
       'v1_driver_age', 'v1_action', 'v1_driver_factors',
       'v1_driver_distracted', 'v1_vehicle_factors', 'v1_most_harmful_event',
       'v1_all_events', 'v2_type', 'v2_driver_age', 'v2_action',
       'v2_driver_factors', 'v2_driver_distracted', 'v2_vehicle_factors',
       'v2_most_harmful_event', 'v2_all_events', 'nonmotorist_factors',
       'factors_roadway', 'lighting', 'hwy_factors', 'pedalcyclist',
       'pedestrian', 'motorcyclist', 'animaltype'],
      dtype='object')

In [36]:
# Split single column with multiple data points into separate columns
traffic_df[['v1_driver_factors_1', 'v1_driver_factors_2','v1_driver_factors_3', 'v1_driver_factors_4']] = traffic_df['v1_driver_factors'].str.split(':', expand=True)

traffic_df[['v1_vehicle_factors_1', 'v1_vehicle_factors_2','v1_vehicle_factors_3', 'v1_vehicle_factors_4', 'v1_vehicle_factors_5']] = traffic_df['v1_vehicle_factors'].str.split(':', expand=True)

traffic_df[['v1_all_events_1', 'v1_all_events_2','v1_all_events_3', 'v1_all_events_4', 'v1_all_events_5']] = traffic_df['v1_all_events'].str.split(':', expand=True)

traffic_df[['v2_driver_factors_1', 'v2_driver_factors_2']] = traffic_df['v2_driver_factors'].str.split(':', expand=True)

traffic_df[['v2_vehicle_factors_1', 'v2_vehicle_factors_2','v2_vehicle_factors_3', 'v2_vehicle_factors_4', 'v2_vehicle_factors_5']] = traffic_df['v2_vehicle_factors'].str.split(':', expand=True)

traffic_df[['v2_all_events_1', 'v2_all_events_2','v2_all_events_3', 'v2_all_events_4', 'v2_all_events_5']] = traffic_df['v1_all_events'].str.split(':', expand=True)

traffic_df[['nonmotorist_factors_1', 'nonmotorist_factors_2','nonmotorist_factors_3', 'nonmotorist_factors_4', 'nonmotorist_factors_5']] = traffic_df['nonmotorist_factors'].str.split(':', expand=True)

traffic_df[['factors_roadway_1', 'factors_roadway_2','factors_roadways_3']] = traffic_df['factors_roadway'].str.split(':', expand=True)

traffic_df[['hwy_factors_1', 'hwy_factors_2','hwy_factors_3', 'hwy_factors_4']] = traffic_df['hwy_factors'].str.split(':', expand=True)

traffic_df.drop(columns = ['v1_driver_factors','v1_vehicle_factors','v1_all_events','v2_driver_factors','v2_vehicle_factors', 'v2_all_events', 'nonmotorist_factors', 'factors_roadway', 'hwy_factors',], inplace = True)

In [37]:
traffic_df['crash_severity'].value_counts()

PROPERTY DAMAGE ONLY    128261
INJURY CRASH             96232
FATAL CRASH               1175
Name: crash_severity, dtype: int64

In [38]:
# Change target values to numeric
# Values are based on Nevada Traffic Records Coordinating Committee Data Dictionary,
# https://zerofatalitiesnv.com/app/uploads/2021/04/2021-01-NV-TRCC-Data-Dictionary.pdf
injury_num = {
    'PROPERTY DAMAGE ONLY': 0,
    'INJURY CRASH': 1,
    'FATAL CRASH': 2,    
}

traffic_df['crash_severity'] = traffic_df['crash_severity'].apply(lambda x: injury_num[x])

traffic_df['crash_severity'].value_counts()

0    128261
1     96232
2      1175
Name: crash_severity, dtype: int64

In [39]:
# Extract datetime values into separate columns
traffic_df['crash_month'] = pd.DatetimeIndex(traffic_df['crash_date']).month
traffic_df['crash_day'] = pd.DatetimeIndex(traffic_df['crash_date']).day
traffic_df['crash_day_of_week'] = pd.DatetimeIndex(traffic_df['crash_date']).dayofweek
traffic_df['crash_hour'] = pd.DatetimeIndex(traffic_df['crash_time']).hour


In [40]:
# Drop original datetime columns
traffic_df.drop(columns = ['crash_date', 'crash_time'], inplace = True)

In [41]:
traffic_df.dtypes

x                    float64
y                    float64
crash_severity         int64
county                object
crash_year             int64
                      ...   
hwy_factors_4         object
crash_month            int64
crash_day              int64
crash_day_of_week      int64
crash_hour             int64
Length: 65, dtype: object

In [42]:
# Get names of all columns with dtypes of "object"
sel_cols = list(traffic_df.select_dtypes(include='object'))

In [43]:
# Preprocessing to convert all strings to numeric values
traffic_df_encoded = pd.get_dummies(traffic_df, columns = sel_cols)
traffic_df_encoded.head()

,x,y,crash_severity,crash_year,total_vehicles,v1_driver_age,v2_driver_age,crash_month,crash_day,crash_day_of_week,...,hwy_factors_3_ OTHER HIGHWAY,hwy_factors_3_ ROAD OBSTRUCTION,"hwy_factors_3_ RUTS, HOLES, BUMPS",hwy_factors_3_ SHOULDERS,hwy_factors_3_ VISUAL OBSTRUCTION(S),hwy_factors_3_ WEATHER,"hwy_factors_3_ WET, ICY, SNOW, SLUSH",hwy_factors_3_ WORK ZONE (CONST. MAINT. UTILITY),hwy_factors_3_ WORN TRAFFIC SURFACE,"hwy_factors_4_ WET, ICY, SNOW, SLUSH"
0,-115.106709,36.236043,0,2016,2,35,41,1,4,0,...,0,0,0,0,0,0,0,0,0,0
1,-119.673794,39.626433,0,2016,1,39,41,1,7,3,...,0,0,0,0,0,0,0,0,0,0
2,-115.101063,36.240200,0,2016,2,31,53,1,4,0,...,0,0,0,0,0,0,0,0,0,0
3,-119.633092,39.516952,0,2016,1,18,41,1,6,2,...,0,0,0,0,0,0,0,0,0,0
4,-115.140570,36.192680,0,2016,1,56,41,1,3,6,...,0,0,0,0,0,0,0,0,0,0


## Model with Crash Severity as Target and All Other Columns as Features

In [44]:
# Separate data into features and target
y = traffic_df_encoded['crash_severity'].values
X = traffic_df_encoded.drop(columns = 'crash_severity')

In [45]:
# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)
X_train.shape

(169251, 1345)

In [46]:
# Scale the model
scaler = StandardScaler() 

# Fit the scaler with the training data
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [47]:
# Create and fit the Random Forest model
rf_model = RandomForestClassifier(n_estimators = 128)
rf_model = rf_model.fit(X_train_scaled, y_train)

In [48]:
# Create predictions
predictions = rf_model.predict(X_test_scaled)
results = pd.DataFrame({
    "Prediction": predictions,
    "Actual": y_test
}).reset_index(drop = True)
results.head()

,Prediction,Actual
0,1,1
1,1,0
2,1,1
3,1,1
4,0,0


In [49]:
# Assess accuracy score
accuracy = accuracy_score(y_test, predictions)
accuracy

0.737667724267508

In [50]:
# Generate confusion matrix and dataframe
cm = confusion_matrix(y_test, predictions)

index_values = ['Actual: Property Damage Only', 
                'Actual: Injury Crash',  
                'Actual: Fatal Crash']

column_values = ['Predicted: Property Damage Only', 
                'Predicted: Injury Crash', 
                'Predicted: Fatal Crash']

cm_df = pd.DataFrame(cm, index = index_values, columns = column_values)
cm_df
                  

,Predicted: Property Damage Only,Predicted: Injury Crash,Predicted: Fatal Crash
Actual: Property Damage Only,25112,6953,0
Actual: Injury Crash,7561,16491,6
Actual: Fatal Crash,66,214,14


## Statewide Results with Crash Severity as Target and All Other Columns as Features

In [60]:
# Display results
print('Statewide Data with Crash Severity as Target')
print('All Other Columns as Features')
print('Confusion Matrix')
display(cm_df)
print(f'\nAccuracy Score: {accuracy}\n')
print('Classification Report')
print(classification_report(y_test, predictions))

Statewide Data with Crash Severity as Target
All Other Columns as Features
Confusion Matrix


,Predicted: Property Damage Only,Predicted: Injury Crash,Predicted: Fatal Crash
Actual: Property Damage Only,25112,6953,0
Actual: Injury Crash,7561,16491,6
Actual: Fatal Crash,66,214,14



Accuracy Score: 0.737667724267508

Classification Report
              precision    recall  f1-score   support

           0       0.77      0.78      0.78     32065
           1       0.70      0.69      0.69     24058
           2       0.70      0.05      0.09       294

    accuracy                           0.74     56417
   macro avg       0.72      0.51      0.52     56417
weighted avg       0.74      0.74      0.74     56417



### Importances with Crash Severity as Target and All Other Columns as Features

In [52]:
# Calculate importance of features
importances = rf_model.feature_importances_
listo = sorted(zip(rf_model.feature_importances_, X.columns), reverse = True)
listo

[(0.05093276587153013, 'y'),
 (0.04982092027700899, 'x'),
 (0.04878507468185123, 'v1_driver_age'),
 (0.045322524526086705, 'v2_driver_age'),
 (0.04116919076391665, 'crash_day'),
 (0.038135058124765954, 'crash_hour'),
 (0.034877688102953705, 'crash_month'),
 (0.030447455949553497, 'crash_day_of_week'),
 (0.02535577942620937, 'lighting_UNKNOWN'),
 (0.022046052544485273, 'factors_roadway_1_DRY'),
 (0.02083477011770424, 'factors_roadway_1_UNKNOWN'),
 (0.02040484264777514, 'crash_year'),
 (0.01840115865688813, 'hwy_factors_1_NONE'),
 (0.015583877571727803, 'hwy_factors_1_UNKNOWN'),
 (0.01433928450768382, 'lighting_DAYLIGHT'),
 (0.013907622736140931, 'total_vehicles'),
 (0.009836293046776133, 'v1_type_SEDAN, 4 DOOR'),
 (0.009276520187369339, 'crash_type_SIDESWIPE, OVERTAKING'),
 (0.00887538770077736, 'v1_vehicle_factors_1_FAILED TO YIELD RIGHT OF WAY'),
 (0.008497395964897353, 'crash_type_ANGLE'),
 (0.008060471416286507, 'v1_action_GOING STRAIGHT'),
 (0.007846749793792237, 'v2_type_SEDAN, 4 

# Second Version of Model: Only Top 50% of Importances as Features

### Importances to Exclude

In [53]:
important = sorted(zip(rf_model.feature_importances_, X.columns), reverse = True)
low_priority = important[772:]

In [54]:
imp_only_list = []
listy_list = []

# iterate using index with enumerate function
for index, tuple in enumerate(low_priority):
       
    # access through index
      # by appending to list
    imp_only_list.append(low_priority[index])
     
# iterate through the list
for x in imp_only_list:
    for y in x:
         listy_list.append(y)

final_list = listy_list[1::2]


### Dataframe without Low-Value Importances

In [55]:
important_df = traffic_df_encoded.drop(columns = final_list)
important_df.head()

,x,y,crash_severity,crash_year,total_vehicles,v1_driver_age,v2_driver_age,crash_month,crash_day,crash_day_of_week,...,hwy_factors_2_ BACKUP DUE TO PRIOR NON-RECURRING INCIDENT,hwy_factors_2_ BACKUP DUE TO REGULAR CONGESTION,hwy_factors_2_ NON-ROADWAY WORK,hwy_factors_2_ OTHER HIGHWAY,"hwy_factors_2_ RUTS, HOLES, BUMPS","hwy_factors_2_ WET, ICY, SNOW, SLUSH",hwy_factors_2_ WORK ZONE (CONST. MAINT. UTILITY),hwy_factors_3_ BACKUP DUE TO REGULAR CONGESTION,hwy_factors_3_ OTHER HIGHWAY,hwy_factors_3_ WORK ZONE (CONST. MAINT. UTILITY)
0,-115.106709,36.236043,0,2016,2,35,41,1,4,0,...,0,0,0,0,0,0,0,0,0,0
1,-119.673794,39.626433,0,2016,1,39,41,1,7,3,...,0,0,0,0,0,0,0,0,0,0
2,-115.101063,36.240200,0,2016,2,31,53,1,4,0,...,0,0,0,0,0,0,0,0,0,0
3,-119.633092,39.516952,0,2016,1,18,41,1,6,2,...,0,0,0,0,0,0,0,0,0,0
4,-115.140570,36.192680,0,2016,1,56,41,1,3,6,...,0,0,0,0,0,0,0,0,0,0


## Model with Crash Severity as Target and Only High-Value Importances as Features

In [56]:
# Separate data into features and target
y_imp = important_df['crash_severity'].values

X_imp = important_df.drop(columns = 'crash_severity')

In [57]:
# had to remove stratify because there was a singleton array
X_train_imp, X_test_imp, y_train_imp, y_test_imp = train_test_split(X_imp, y_imp)
X_train_imp.shape

(169251, 772)

In [58]:
# Scale the model
scaler_imp = StandardScaler() 

# Fit the scaler with the training data
X_scaler_imp = scaler_imp.fit(X_train_imp)
X_train_scaled_imp = X_scaler_imp.transform(X_train_imp)
X_test_scaled_imp = X_scaler_imp.transform(X_test_imp)

In [59]:
# Create and fit the Random Forest model
rf_model_imp = RandomForestClassifier(n_estimators = 128)
rf_model_imp = rf_model.fit(X_train_scaled_imp, y_train_imp)

KeyboardInterrupt: 

In [ ]:
# Create predictions
predictions_imp = rf_model_imp.predict(X_test_scaled_imp)
results_imp = pd.DataFrame({
    "Prediction": predictions_imp,
    "Actual": y_test_imp
}).reset_index(drop = True)
results_imp.head()

In [ ]:
# Assess accuracy score
accuracy_imp = accuracy_score(y_test_imp, predictions_imp)
accuracy_imp

In [ ]:
# Generate confusion matrix and dataframe
cm_imp = confusion_matrix(y_test_imp, predictions_imp)

index_values_imp = ['Actual: Property Damage Only', 
                'Actual: Injury Crash',  
                'Actual: Fatal Crash']

column_values_imp = ['Predicted: Property Damage Only', 
                'Predicted: Injury Crash', 
                'Predicted: Fatal Crash']

cm_df_imp = pd.DataFrame(cm_imp, index = index_values_imp, columns = column_values_imp)
cm_df_imp

## Statewide Results with Crash Severity as Target and High-Priority Columns as Features

In [ ]:
# Display results
print('Statewide Data with Crash Severity as Target, High-Priority Columns as Features')
print('Confusion Matrix')
display(cm_df_imp)
print(f'\nAccuracy Score: {accuracy_imp}\n')
print('Classification Report')
print(classification_report(y_test_imp, predictions_imp))

### Calculate Importance of Features

In [ ]:
importances_high_priority = rf_model_imp.feature_importances_
sorted(zip(importances_high_priority, X_imp.columns), reverse = True)

# Third Version of Model with Crash Severity as Target and Only Importances >= 0.01

In [ ]:
lowest_priority = important[17:]

In [ ]:
imp_low_list = []
exclude_list = []

# iterate using index with enumerate function
for index, tuple in enumerate(lowest_priority):
       
    # access through index by appending to list
    imp_low_list.append(lowest_priority[index])
     
# iterate through the list
for x in imp_low_list:
    for y in x:
        exclude_list.append(y)

final_exclude_list = exclude_list[1::2]


In [ ]:
imp_top_df = traffic_df_encoded.drop(columns = final_exclude_list)
imp_top_df.head()

## Model with Crash Type as Target and Only Importances >= 0.01 as Features

In [ ]:
# Separate data into features and target
y_imp_top = imp_top_df['crash_type'].values
X_imp_top = imp_top_df.drop(columns = 'crash_type')

In [ ]:
X_train_imp_top, X_test_imp_top, y_train_imp_top, y_test_imp_top = train_test_split(X_imp_top, y_imp_top)
X_train_imp_top.shape

In [ ]:
# Scale the model
scaler_imp_top = StandardScaler() 

# Fit the scaler with the training data
X_scaler_imp_top = scaler_imp_top.fit(X_train_imp_top)
X_train_scaled_imp_top = X_scaler_imp_top.transform(X_train_imp_top)
X_test_scaled_imp_top = X_scaler_imp_top.transform(X_test_imp_top)

In [ ]:
# Create and fit the Random Forest model
rf_model_imp_top = RandomForestClassifier(n_estimators = 128)
rf_model_imp_top = rf_model_imp_top.fit(X_train_scaled_imp_top, y_train_imp_top)

In [ ]:
# Create predictions
predictions_imp_top = rf_model_imp_top.predict(X_test_scaled_imp_top)
results_imp_top = pd.DataFrame({
    "Prediction": predictions_imp_top,
    "Actual": y_test_imp_top
}).reset_index(drop = True)
results_imp_top.head()

In [ ]:
# Assess accuracy score
accuracy_imp_top = accuracy_score(y_test_imp_top, predictions_imp_top)
accuracy_imp_top

In [ ]:
# Generate confusion matrix and dataframe
cm_imp_top = confusion_matrix(y_test_imp_top, predictions_imp_top)

index_values_imp = ['Actual: Unknown', 
                'Actual: Property Damage Only', 
                'Actual: Possible Injury', 
                'Actual: Suspected Minor Injury', 
                'Actual: Suspected Serious Injury', 
                'Actual: Fatal Injury']

column_values_imp = ['Predicted: Unknown', 
                'Predicted: Property Damage Only', 
                'Predicted: Possible Injury', 
                'Predicted: Suspected Minor Injury', 
                'Predicted: Suspected Serious Injury', 
                'Predicted: Fatal Injury']

cm_df_imp_top = pd.DataFrame(cm_imp_top, index = index_values_imp, columns = column_values_imp)
cm_df_imp_top

## Statewide Results with Crash Type as Target and Only Importances >= 0.01 as Features

In [ ]:
# Display results
print('Statewide Data with Crash Type as Target, Only Top 17 as Features')
print('Confusion Matrix')
display(cm_df_imp_top)
print(f'\nAccuracy Score: {accuracy_imp_top}\n')
print('Classification Report')
print(classification_report(y_test_imp_top, predictions_imp_top))